In [ ]:
import numpy as np
from collections import deque

class Node:
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

class Puzzle:
    def __init__(self, start, goal):
        self.start = (tuple(map(tuple, start)), self.find_empty(start))  # Dynamically find empty space
        self.goal = (tuple(map(tuple, goal)), self.find_empty(goal))
        self.solution = None
        self.num_explored = 0
        self.total_states_generated = 0  # Counter for total states generated

    def find_empty(self, state):
        # Find the position of the empty space (0)
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    return (i, j)

    def neighbors(self, state):
        mat, (row, col) = state
        results = []
        directions = [(1, 0, 'down'), (-1, 0, 'up'), (0, 1, 'right'), (0, -1, 'left')]

        for dr, dc, action in directions:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 a jind 0 <= new_col < 3:
                mat1 = np.copy(mat)
                # Swap the empty space (0) with the adjacent number
                mat1[row][col], mat1[new_row][new_col] = mat1[new_row][new_col], mat1[row][col]
                results.append((action, (tuple(map(tuple, mat1)), (new_row, new_col))))
        return results

    def solve(self):
        start_node = Node(state=self.start, parent=None, action=None)
        queue = deque([start_node])  # Use deque for efficient pops from the left
        explored = set()  # Use a set for explored states

        while queue:
            node = queue.popleft()  # Dequeue the first node
            self.num_explored += 1

            # Check if the current node's state is the goal
            if node.state[0] == self.goal[0]:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return  # Found a solution

            # Mark the state as explored
            explored.add(node.state[0])

            # Explore neighbors
            for action, state in self.neighbors(node.state):
                if state[0] not in explored and all(node.state[0] != state[0] for node in queue):
                    child = Node(state=state, parent=node, action=action)
                    queue.append(child)  # Enqueue the child node
                    self.total_states_generated += 1  # Increment the total states generated counter

    def print_solution(self):
        if self.solution is None:
            print("No solution found.")
            return

        print("Start State:\n", np.array(self.start[0]), "\n")
        print("Goal State:\n", np.array(self.goal[0]), "\n")
        print("\nStates Explored: ", self.num_explored)
        print("Total States Generated: ", self.total_states_generated, "\n")

        print("Actions Taken to Reach the Goal:\n")
        for action, cell in zip(self.solution[0], self.solution[1]):
            print("Action: ", action)
            print(np.array(cell[0]), "\n")
        print("Goal Reached!!")

# Example usage
start = np.array([[1, 2, 3], [0, 4, 6], [7, 5, 8]])
goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

p = Puzzle(start, goal)
p.solve()
p.print_solution()

Start State:
 [[1 2 3]
 [0 4 6]
 [7 5 8]] 

Goal State:
 [[1 2 3]
 [4 5 6]
 [7 8 0]] 


States Explored:  14
Total States Generated:  26 

Actions Taken to Reach the Goal:

Action:  right
[[1 2 3]
 [4 0 6]
 [7 5 8]] 

Action:  down
[[1 2 3]
 [4 5 6]
 [7 0 8]] 

Action:  right
[[1 2 3]
 [4 5 6]
 [7 8 0]] 

Goal Reached!!


In [ ]:
import numpy as np #dfs


class Node:
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action


class StackFrontier:
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any((node.state[0] == state[0]).all() for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("Empty Frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node


class Puzzle:
    def __init__(self, start, startIndex, goal, goalIndex):
        self.start = [start, startIndex]
        self.goal = [goal, goalIndex]
        self.solution = None

    def neighbors(self, state):
        mat, (row, col) = state
        results = []

        if row > 0:  # Move up
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row - 1][col]
            mat1[row - 1][col] = 0
            results.append(('up', [mat1, (row - 1, col)]))
        if col > 0:  # Move left
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row][col - 1]
            mat1[row][col - 1] = 0
            results.append(('left', [mat1, (row, col - 1)]))
        if row < 2:  # Move down
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row + 1][col]
            mat1[row + 1][col] = 0
            results.append(('down', [mat1, (row + 1, col)]))
        if col < 2:  # Move right
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row][col + 1]
            mat1[row][col + 1] = 0
            results.append(('right', [mat1, (row, col + 1)]))

        return results

    def print_solution(self):
        if self.solution is not None:
            print("Start State:\n", self.start[0], "\n")
            print("Goal State:\n", self.goal[0], "\n")
            print("\nStates Explored: ", self.num_explored, "\n")
            print("Solution:\n ")
            for action, cell in zip(self.solution[0], self.solution[1]):
                print("Action: ", action, "\n", cell[0], "\n")
            print("Goal Reached!!")
        else:
            print("No solution found.")

    def solve(self):
        self.num_explored = 0
        start = Node(state=self.start, parent=None, action=None)
        frontier = StackFrontier()  # Use StackFrontier for DFS
        frontier.add(start)

        self.explored = set()  # Use a set to track explored states

        while True:
            if frontier.empty():
                raise Exception("No solution")

            node = frontier.remove()
            self.num_explored += 1

            # Display the explored state and the action taken to get there
            if node.parent is not None:
                print(f"Exploring state after moving '{node.action}':\n{node.state[0]}\n")

            if (node.state[0] == self.goal[0]).all():
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return

            # Convert state to a tuple for set operations
            state_tuple = tuple(map(tuple, node.state[0]))
            if state_tuple not in self.explored:
                self.explored.add(state_tuple)  # Add to explored set

                for action, state in self.neighbors(node.state):
                    child_state_tuple = tuple(map(tuple, state[0]))
                    if not frontier.contains_state(state) and child_state_tuple not in self.explored:
                        child = Node(state=state, parent=node, action=action)
                        frontier.add(child)


def input_puzzle():
    print("Enter the initial state (3x3 matrix, use spaces to separate numbers):")
    start = np.array([list(map(int, input().split())) for _ in range(3)])

    print("Enter the goal state (3x3 matrix, use spaces to separate numbers):")
    goal = np.array([list(map(int, input().split())) for _ in range(3)])

    startIndex = tuple(map(int, np.argwhere(start == 0)[0]))  # Find position of 0 in start
    goalIndex = tuple(map(int, np.argwhere(goal == 0)[0]))  # Find position of 0 in goal

    return start, startIndex, goal, goalIndex


if __name__ == "__main__":
    start, startIndex, goal, goalIndex = input_puzzle()
    p = Puzzle(start, startIndex, goal, goalIndex)
    p.solve()
    p.print_solution()

Streaming output truncated to the last 5000 lines.
Exploring state after moving 'up':
[[5 7 4]
 [0 6 2]
 [1 3 8]]

Exploring state after moving 'up':
[[0 7 4]
 [5 6 2]
 [1 3 8]]

Exploring state after moving 'right':
[[7 0 4]
 [5 6 2]
 [1 3 8]]

Exploring state after moving 'right':
[[7 4 0]
 [5 6 2]
 [1 3 8]]

Exploring state after moving 'down':
[[7 4 2]
 [5 6 0]
 [1 3 8]]

Exploring state after moving 'down':
[[7 4 2]
 [5 6 8]
 [1 3 0]]

Exploring state after moving 'left':
[[7 4 2]
 [5 6 8]
 [1 0 3]]

Exploring state after moving 'left':
[[7 4 2]
 [5 6 8]
 [0 1 3]]

Exploring state after moving 'up':
[[7 4 2]
 [0 6 8]
 [5 1 3]]

Exploring state after moving 'up':
[[0 4 2]
 [7 6 8]
 [5 1 3]]

Exploring state after moving 'right':
[[4 0 2]
 [7 6 8]
 [5 1 3]]

Exploring state after moving 'right':
[[4 2 0]
 [7 6 8]
 [5 1 3]]

Exploring state after moving 'down':
[[4 2 8]
 [7 6 0]
 [5 1 3]]

Exploring state after moving 'down':
[[4 2 8]
 [7 6 3]
 [5 1 0]]

Exploring state after moving 